# Ring Throw Parameter Sweeping

This notebook demonstrates how to:
1. Load the robotics environment with RAI.
2. Parameterize certain variables in the ring-throw code.
3. Loop over a range of values for those parameters.
4. Run the simulation for each combination.
5. Log the outcomes to a CSV.


In [ ]:
import robotic as ry
import numpy as np
import os

# Set the DISPLAY environment variable
os.environ['DISPLAY'] = ':0'

# Global environment setup
C = ry.Config()
C.addFile("/home/yagz/rai_venv/robotics/penvironment.g")  # Adjust path if needed
C.view(False)
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

# -----------------------------
#  Utility & Helper Functions
# -----------------------------

def align_gripper_to_ring():
    approach_waypoint = C.addFrame('approach_waypoint', 'ring1')  # Using ring1 for alignment
    align_waypoint = C.addFrame('align_waypoint', 'ring1')
    approach_waypoint.setShape(ry.ST.marker, size=[.1])
    approach_waypoint.setRelativePose('t(0 0 0.1) d(90 0 0 0)')  # Position above ring1

    align_waypoint.setShape(ry.ST.marker, size=[.1])
    align_waypoint.setRelativePose(' d(90 0 0 0)')  # Align directly to ring1

    komo = ry.KOMO()
    komo.setConfig(C, True)
    komo.setTiming(2., 1, 5., 0)
    komo.addControlObjective([], 0, 1e-0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([1.], ry.FS.positionDiff, ['l_gripper', 'approach_waypoint'], ry.OT.eq, [1e2])
    komo.addObjective([2.], ry.FS.positionDiff, ['l_gripper', 'align_waypoint'], ry.OT.eq, [1e2])

    solver = ry.NLP_Solver()
    solver.setProblem(komo.nlp())
    solver.setOptions(stopTolerance=1e-2, verbose=4)
    solver.solve()

    return komo.getPath()

def grasp_ring():
    bot.gripperCloseGrasp(ry._left, 'ring1', force=10.0, width=0, speed=1)
    while not bot.gripperDone(ry._left):
        bot.sync(C, 0.1)
    print("Ring successfully grasped by the gripper.")

def strech():
    strech_frame1 = C.addFrame('strech_frame1', 'table')
    strech_frame1.setShape(ry.ST.marker, size=[.1])
    strech_frame1.setRelativePosition([0, 2.5, 0.5])

    komo = ry.KOMO()
    komo.setConfig(C, True)
    komo.setTiming(2., 1, 5., 1)
    komo.addControlObjective([], 0, 1e-0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([1.], ry.FS.positionDiff, ['l_gripper', 'strech_frame1'], ry.OT.eq, [1e1], order=0)

    solver = ry.NLP_Solver()
    solver.setProblem(komo.nlp())
    solver.setOptions(stopTolerance=1e-2, verbose=4)
    solver.solve()

    return komo.getPath()

def throw():
    rel_point = [0, 1.6, 1]
    komo = ry.KOMO()
    komo.setConfig(C, True)
    komo.setTiming(2., 1, 5., 1)
    komo.addControlObjective([], 0, 1e-0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

    # Position objective
    komo.addObjective([1.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1], target=rel_point)
    # Velocity objective
    komo.addObjective([1.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1],
                      target=[0, -3.4, 8.355], order=1)

    solver = ry.NLP_Solver()
    solver.setProblem(komo.nlp())
    solver.setOptions(stopTolerance=1e-2, verbose=4)
    solver.solve()

    return komo.getPath()

# Placeholder for success detection logic
def is_ring_on_pole():
    """
    Implement your real check if ring ended on the pole.
    Currently returns False always.
    """
    return False

# -----------------------------
#  Core Parametrized Function
# -----------------------------
def run_ring_throw(q2_val, time1, time2, release_threshold):
    """
    Executes the ring pick-and-throw sequence, using:
      - q2_val in [-0.51..-0.11] for q[2]
      - (time1, time2) in ([0..0.2], [1..1.5]) for the throw move
      - release_threshold in [0.4..0.7]

    Returns True if 'ring on pole' success, else False.
    """

    # 1) Align with the ring
    grasp_path = align_gripper_to_ring()
    # keep the align times as [1., 1.5]
    bot.move(grasp_path, [1.0, 1.5])
    # If you want to see the motion without waiting too long, reduce the second param or skip wait:
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # 2) Grasp ring
    grasp_ring()
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # 3) Adjust q[2] & q[3]
    q = bot.get_qHome()
    q[2] = q2_val          # param in [-0.51..-0.11]
    q[3] = q[3] + 4        # same as original code
    bot.moveTo(q)
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # 4) Compute throw path
    throw_path = throw()
    bot.move(throw_path, [time1, time2])  # param in [time1, time2]

    # 5) Release logic
    while bot.getTimeToEnd() > 0:
        bot.sync(C, 0.05)
        gripper_position = C.frame("l_gripper").getPosition()
        gripper_height = gripper_position[2]

        # if gripper_height > release_threshold => open gripper
        if gripper_height > release_threshold:
            bot.gripperMove(ry._left, width=10, speed=10)

    # If you want the block to end quickly, you can skip the final wait or shorten it:
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # Evaluate success
    success = is_ring_on_pole()
    return success

# Example single-run test
def test_run():
    """
    Single-run test without blocking input().
    """
    print("Running a quick test throw...")
    result = run_ring_throw(
        q2_val=-0.31,
        time1=0.1,
        time2=1.2,
        release_threshold=0.52
    )
    print("Test run success?", result)

# We'll just call the test_run() function at the end of the cell.
test_run()
print("Block completed. If the cell still runs indefinitely, remove or shorten wait calls.")


-- WARNING:graph.cpp:edit:494(-1) no nodes edited! (from 'panda_finger_joint1: { joint_active! }')
-- WARNING:kin_physx.cpp:prepareLinkShapes:625(-1) computing compound inertia for object frame 'ring' -- this should have been done earlier?
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):5966.48 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):     3200.5  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    1793.12  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    898.333  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    346.398  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    68.2359  ACCEPT
--newton-- it:   6  |Delta|:   0.152383  alpha:          1  evals:   7  f(y):    13.2498  ACCEPT
--newton-- it:   7  |Delta|: 

## 2) Parameter Sweeping (Runner)

We will define the ranges for:
- **`q2_val`** in `[-0.51..-0.11]`
- **`time1`** in `[0..0.2]`
- **`time2`** in `[1..1.5]`
- **`release_threshold`** in `[0.4..0.7]`

Then we will loop over them, run the sim, and record the results in a CSV.

In [2]:
import csv

# Let's define some step sizes for each parameter
q2_vals = np.linspace(-0.51, -0.11, 5)           # e.g. 5 steps
time1_vals = np.linspace(0.0, 0.2, 3)            # e.g. 0.0, 0.1, 0.2
time2_vals = np.linspace(1.0, 1.5, 3)            # e.g. 1.0, 1.25, 1.5
release_threshes = np.linspace(0.4, 0.7, 4)      # e.g. 0.4, 0.5, 0.6, 0.7

output_file = "throw_results.csv"

with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    # CSV header
    writer.writerow(["q2_val", "time1", "time2", "release_threshold", "success"])

    # nested loops
    for q2 in q2_vals:
        for t1 in time1_vals:
            for t2 in time2_vals:
                for rel_th in release_threshes:
                    print(f"Running with q2={q2:.2f}, t1={t1:.2f}, t2={t2:.2f}, rel_th={rel_th:.2f}")

                    success = run_ring_throw(
                        q2_val = q2,
                        time1 = t1,
                        time2 = t2,
                        release_threshold = rel_th
                    )

                    writer.writerow([q2, t1, t2, rel_th, success])

print("\nAll simulations complete. Results saved in", output_file)


Running with q2=-0.51, t1=0.00, t2=1.00, rel_th=0.40
-- WARNING:kin.cpp:addFrame:193(-1) frame already exists! returning existing without modifications!
-- WARNING:kin.cpp:addFrame:193(-1) frame already exists! returning existing without modifications!
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):110498 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):     102864  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    95744.5  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    89249.4  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    83539.4  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    78800.4  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    75381.7  ACCEPT
--newton-- it:   

KeyboardInterrupt: 